<a href="https://colab.research.google.com/github/worldwidekatie/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

### Data Cleaning and Loading

In [100]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', 
                 names=['party', 'handicapped-infants', 'water-project', 
                        'budget', 'physician-fee-freeze', 'el-salvador-aid', 
                        'religious-groups', 'anti-satellite-ban', 
                        'aid-to-contras', 'mx-missile', 'immigration', 
                        'synfuels', 'education', 'right-to-sue', 
                        'crime', 'duty-free', 'south-africa'])

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [101]:
import numpy as np
df = df.replace({"y": 1, "n": 0, "?": np.NaN})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [102]:
rep = df[df['party']=='republican']

rep.head()
rep.mean()

handicapped-infants     0.187879
water-project           0.506757
budget                  0.134146
physician-fee-freeze    0.987879
el-salvador-aid         0.951515
religious-groups        0.897590
anti-satellite-ban      0.240741
aid-to-contras          0.152866
mx-missile              0.115152
immigration             0.557576
synfuels                0.132075
education               0.870968
right-to-sue            0.860759
crime                   0.981366
duty-free               0.089744
south-africa            0.657534
dtype: float64

In [103]:
dem = df[df['party']=='democrat']

dem.head()
dem.mean()

handicapped-infants     0.604651
water-project           0.502092
budget                  0.888462
physician-fee-freeze    0.054054
el-salvador-aid         0.215686
religious-groups        0.476744
anti-satellite-ban      0.772201
aid-to-contras          0.828897
mx-missile              0.758065
immigration             0.471483
synfuels                0.505882
education               0.144578
right-to-sue            0.289683
crime                   0.350195
duty-free               0.637450
south-africa            0.935135
dtype: float64

### Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

1) Null Hypothesis: Democrat support for the budget is equal to republican support for the budget.

$H_0: \bar{x}_{d} == \bar{x}_{r}$

2) Alternative Hypothesis: Democrat support for the budget is not equal to republican support for the budget

$H_a: \bar{x}_{d} \neq \bar{x}_{r}$

3) Confidence Level: 95%

In [0]:
#Budget
from scipy import stats

stats.ttest_ind(dem['budget'], rep['budget'], nan_policy='omit')


Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

### Conclusions

Based on a t-statistic of 23.21 and a p-value of 2.07e-77, I reject the null hypothesis that Democrats and Republicans support the budget equally and instead offer an alternative hypothesis, that Democrats and Republicans do not support the budget at an equal rate.

### Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

1) Null Hypothesis: Republican support for the "physician fee freeze" bill is equal to Democrat support for the "physician fee freeze" bill.

$H_0: \bar{x}_{r} == \bar{x}_{d}$

2) Alternative Hypothesis: Republican support for the "physician fee freeze" bill is not equal to Democratic support for the "physician fee freeze" bill

$H_a: \bar{x}_{r} \neq \bar{x}_{d}$

3) Confidence Level: 95%

In [0]:
#Physician fee freeze
stats.ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

## Conclusions

Based on a t-statistic of 49.37 and a p-value of 1.99c-177, I reject the null hypothesis that Republicans and Democrats support the physician fee freeze bill at the same rate and instead offer an alternative hypothesis, that they support the bill at a different rate.

### Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

1) Null Hypothesis: Democrat support for the "water project" bill is equal to Republican support for the "water project" bill.

$H_0: \bar{x}_{r} == \bar{x}_{d}$

2) Alternative Hypothesis: Democrat support for the "water project" bill is not equal to Republican support for the "water project" bill.

$H_a: \bar{x}_{r} \neq \bar{x}_{d}$

3) Confidence Level: 95%

In [0]:
#Water project
stats.ttest_ind(dem['water-project'], rep['water-project'], nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

Based on a t-value of -0.09 and a p-value of 0.93, I fail to reject the null hypothesis that Republicans and Democrats support the water project at the same rate.

# Stretch Goals
## Refactor your code into functions so it's easy to rerun with arbitrary variables.

In [0]:
#My Function
def Ttest(df1, df2):
  return stats.ttest_ind(df1, df2, nan_policy='omit')

In [0]:
Ttest(dem['budget'], rep['budget'])

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
#My Function + Means
def Ttest(df1, df2):
  print("df1 mean")
  print(df1.mean())
  print(" ")
  print("df2 mean")
  print(df2.mean())
  print(" ")
  return stats.ttest_ind(df1, df2, nan_policy='omit')

In [113]:
Ttest(dem['budget'], rep['budget'])

df1 mean
0.8884615384615384
 
df2 mean
0.13414634146341464
 


Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
#My Function with a for loop
def Ttest_loop(df1, df2):
  issues = df1.columns.tolist()
  issues.remove('party')
  output=[]

  for issue in issues:
    output.append(stats.ttest_ind(df1[issue], df2[issue], nan_policy='omit'))
  return output

In [0]:
#I can't believe that worked. It's blowing my mind.
Ttest_loop(dem, rep)

[Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18),
 Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485),
 Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77),
 Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177),
 Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68),
 Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20),
 Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31),
 Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54),
 Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47),
 Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066),
 Ttest_indResult(statistic=8.293603989407588, pvalue=1.5759322301054064e-15),
 Ttest_indResult(statistic=-20.500685724563073, pvalue=1.8834203990450192e-64),
 Ttest_indResult(statistic=-13.51064251060933, pvalue=1.227858170

In [0]:
#My Function with a for loop with added info
import pandas as pd

def Ttest_loop(df1, df2):
  issues = df1.columns.tolist()
  issues.remove('party')
  output=[]

  for issue in issues:
    output.append([issue, "df1 mean", df1[issue].mean(), "df2 mean", df2[issue].mean(), stats.ttest_ind(df1[issue], df2[issue], nan_policy='omit')])
   
  return output


In [130]:
#OMG... I can't believe I got that to work!
Ttest_loop(dem, rep)

[['handicapped-infants',
  'df1 mean',
  0.6046511627906976,
  'df2 mean',
  0.18787878787878787,
  Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)],
 ['water-project',
  'df1 mean',
  0.502092050209205,
  'df2 mean',
  0.5067567567567568,
  Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)],
 ['budget',
  'df1 mean',
  0.8884615384615384,
  'df2 mean',
  0.13414634146341464,
  Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)],
 ['physician-fee-freeze',
  'df1 mean',
  0.05405405405405406,
  'df2 mean',
  0.9878787878787879,
  Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)],
 ['el-salvador-aid',
  'df1 mean',
  0.21568627450980393,
  'df2 mean',
  0.9515151515151515,
  Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)],
 ['religious-groups',
  'df1 mean',
  0.47674418604651164,
  'df2 mean',
  0.8975903614457831,
  Ttest_indResult(statistic=-9.737575825

## Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.

### Start with a 1-sample T-test

**- Establish the conditions for your test**
1. Null Hypothesis: Democrats support the "education" bill is split 50/50: $H_0: \mu == .5$

2. Alternative Hypothesis: Democratic support for the "education bill" is not 50/50: $H_a: \mu \neq .5$

3. Confidence Level: 95%


**Calculate the T-Statistic**


In [0]:
from IPython.display import display, Image
url = 'https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png'

one_sample = Image(url=url, width=400)

display(one_sample)

dem['education'].describe()

count    249.000000
mean       0.144578
std        0.352383
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: education, dtype: float64

In [0]:
import math
(0.144578-0.5) / (0.352383/math.sqrt(249))

-15.915820457260377

In [0]:
#Checking my work with SciPy
stats.ttest_1samp(dem['education'], .5, nan_policy='omit')

Ttest_1sampResult(statistic=-15.915784824279635, pvalue=8.900198902856228e-40)

###Conclusions

Based on a t-statistic of -15.92 and a p-value of 8.9e-40, I reject the null hypothesis that Democrats support the "education" bill at a 50/50 level and instead offer an alternative hypothesis, that Democrats do not support the "education" bill at a 50/50 level.

## Then try a 2-sample T-Test

**- Establish the conditions for your test**
1. Null Hypothesis: Republicans and Democrats support the "crime" bill at equal levels: $H_0: \bar{x}_{r} == \bar{x}_{d}$

2. Alternative Hypothesis: Republicans and Democrats do not support the "crime" bill at equal levels: $H_0: \bar{x}_{r} \neq \bar{x}_{d}$

3. Confidence Level: 95%


**Calculate the T-Statistic**

In [0]:
from IPython.display import display, Image
url = 'https://www.statsdirect.co.uk/help/generatedimages/equations/equation167.svg'

two_sample = Image(url=url, width=400)

display(two_sample)

print("Dem Crime")
print(dem['crime'].describe())
print("Rep Crime")
print(rep['crime'].describe())
print("for pooled sample variance")
df['crime'].describe()

Dem Crime
count    257.000000
mean       0.350195
std        0.477962
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: crime, dtype: float64
Rep Crime
count    161.000000
mean       0.981366
std        0.135649
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: crime, dtype: float64
for pooled sample variance


count    418.000000
mean       0.593301
std        0.491806
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: crime, dtype: float64

In [0]:
#Calculate the t-statistic
(0.35-0.98) / math.sqrt((0.49**2) * (1/257 + 1/161))

#And the University of Iowa Applette is giving me 0 as a p-value

-12.791922754803782

In [0]:
#Check my work with scipy. I did not get the correct t-statistic. 
#Maybe it's because I couldn't figure out how to do S-squared correctly.
#The equation for that was too much of a beast for me to figure out.
#I am thankful that never during the course of my career have I
#asked by a client to do that by hand.
stats.ttest_ind(rep['crime'], dem['crime'], nan_policy='omit')

Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)

###Conclusions

Based on a t-statistic of 16.34 and a p-value of 9.95e-47, I reject the null hypothesis that Republicans and Democrats have equal support for the "crime" bill and instead offer an alternative hypothesis that Republicans an Democrats do not support the crime bill at equal levels.

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!